In [10]:
import logging

from Loader import fileloader_proto as fl
from Loader import datahandledatabase as dhdb
from Loader import datacompute as dc
from Loader import datapreparation as dp
from Loader import dfutils as utils
import pandas as pd
import numpy as np
import posixpath
from datetime import datetime
import time
import xlwings as xw
import copy

logger = logging.getLogger("")
logger.setLevel(logging.INFO)

In [11]:
month = '201905'#para cargar unitarios, disminuir 1 mes
month_unitario = 201902#poner un mes mas que el de arriba.

inifile = fl.ReadIniFile(mercado="empresas")
parser = inifile.getIniFileParser()

# Configurando los archivos de configuración para administrar la base de datos
dbparser = inifile.getDbIniFileParser()
dbmanager = dhdb.DbDataProcess(month)
dbmanager.setParser(dbparser)


INFO:    Values loaded from globals.ini:
INFO:     * mainpath C:/Users/Johnny/OneDrive - Entel Peru S.A
INFO:     * scriptspath C:/Users/Johnny/Documents/Comisiones/Comisiones-python
INFO:     * databasepath C:/Users/Johnny/OneDrive - Entel Peru S.A/SQLiteDB
INFO:    datapath value is C:/Users/Johnny/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/xlsx
INFO:    testpath value is C:/Users/Johnny/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/test
INFO:    Setting database path to C:/Users/Johnny/OneDrive - Entel Peru S.A/SQLiteDB/mercado_empresas_db.sqlite


In [7]:
# Configurando los archivos de configuración para importar archivos tipo : xlsx, txt, ini
parser = inifile.getIniFileParser()

loader = fl.LoadFileProcess(month)
loader.setParser(parser)
loader.setDefaultPath(defaultpath)

# Carga de Archivos
#*******************
# Asegurate que el directorio donde está el archivo este correcto
# Asegurate que no haya pestañas ocultas en Bases GCP ni Plataformas. 
# En GCP_Base comisiones Partir en dos periodos


# 
section_empleados_update = fl.SectionObj(parser,"Empleados_Update",month)
loader2 = fl.LoadFileProcess(section_empleados_update)
nuevospuestos = loader2.loadFile()

# Preparación : Ninguna
# File :  _RRHH Ceses
#ceses = loader.loadFile('Ceses')

# Preparación : Importar en periodo Mensual y quincenal
# Advertencia : Verificar que no se duplique
# File :  _Soporte Estructura Canales Venta Mercado Empresa
#jerarquia = loader.loadFile('Jerarquia')

# Preparación : Agregar la columna de Seguro si Gestión de Información no lo proporciona
# File :  _Planeamiento Inar Empresas
#inar = loader.loadFile('Inar')

# Preparación :Completar dos columnas adicionales ver archivo anterior. Mensual y Quincenal
# File : _Planeamiento Deacs Empresas
#deacs = loader.loadFile('Deacs')

# Preparación : Completar dos columnas y Verificar Fecha. Periodo Mensual y Quincenal. Completar  vendedor_cross_selling 
# de _GCP BASE COMISIONES (BASE SVA / VENDEDOR PYME). Partir el GCE RAS en Ventas_SSAA y Deacs_SSAA
# File : _GCE Ventas SSAA
#ventas_ssaa = loader.loadFile('Ventas_SSAA')  # listo para subir

#ventas_ssaa = loader.loadFile('Ventas_SSAA_new')  # listo para subir




# Preparación : Completar dos columnas.Verificar Fechas. Periodo Mensual y Quincenal.
# File : _GCE Deacs SSAA
#deacs_ssaa = loader.loadFile('Deacs_SSAA') #listo para subir

# Preparación : Ingresar en Periodo Mensual y Quincenal. Verificar Fechas
# File : _GCE Paquetes
#paquetes = loader.loadFile('Paquetes_new') #listo para subir

# Advertencia : la lectura del unitario es en el tiempo Mes de Comisiones -1. 
# File :  _Comisiones Pymes
#unitarios = loader.loadFile('Unitarios') 
#unitarios = unitarios[unitarios['COMISION_UNITARIA']>0]
#unitarios.reset_index(drop = True, inplace = True)
#soluciones de negocio en COMISION_UNITARIA, VENDEDOR, FECHA_PROCESO

# Preparación : Completar columnas numericas Cero
# File : _Experiencia Bolsa 3G
#bolsas = loader.loadFile('Bolsas') #listo para subir

# Preparación : Ninguna
# File : _Riesgos Actividad Lima, _Riesgos Actividad Regiones
#actividad = loader.loadFile('Actividad') 

# Preparación : Update despues de Ingresar el INAR mensual. Llenar carterización y dealer regiones
# File _ : _GCP Base Comisiones
#basesgcp = loader.loadFile('Bases_GCP') 

# Advertencia : Separar en dos Periodos debido a Equipos de Captura Quincenal, tomar sólo el quincenal del periodo anterior
# File : _GCP Base Quincenal Comisiones
#basesgcpquincenal = loader.loadFile('Bases_GCP_Quincenal') #no se esta usando

# Preparación : Ver Errores. Cargar a conveniencia
# Advertencia : Importar en caso no lo incluya Gestión de Información
# File : _GCE Base Comisiones
#basesgce = loader.loadFile('Bases_GCE') #no se esta usando

INFO:    Values loaded from globals.ini:
INFO:    Values loaded from globals.ini:


El tamaño de Empleados_Update es 1 registros
El tamaño de Paquetes_new es 257 registros


In [22]:
# Testing Dataframes

#jerarquia.dtypes

#inar.dtypes
#deacs.dtypes 

#ventas_ssaa.dtypes
#ventas_ssaa_new.dtypes
#deacs_ssaa.dtypes
#paquetes.dtypes
#bolsas.dtypes

#basesgcp.dtypes
#basesgcp.describe()
#basesgcp.describe(include = ['O'])
#basesgce.dtypes
#actividad.dtypes
#basesgcpquincenal.dtypes
#jerarquia.dtypes

#unitarios.dtypes

# Looking the df
unitarios
#unitarios[unitarios['PERIODO_ACTIVACION']==201901]
#basesgcp[basesgcp['SEGMENTO_ACCESS'].notnull()].head()
#paquetes.head()
#ceses.head()
#basesgcp


,PERIODO_ACTIVACION,ZONAVENTA,RAZON_SOCIAL,CONTRATO,FECHA_PROCESO,ESTADO,VENDEDOR,COMISION_UNITARIA
0,201903,DNEGOCIOS_EMPRESAS1,GRUPO ESPACIO URBANO S.A.C.,48691253.0,43530,NEWS,CCARRILLOP,63.25
1,201903,DNEGOCIOS_EMPRESAS1,GRUPO ESPACIO URBANO S.A.C.,48691294.0,43530,NEWS,CCARRILLOP,63.25
2,201903,DNEGOCIOS_EMPRESAS1,GRUPO ESPACIO URBANO S.A.C.,48691331.0,43530,NEWS,CCARRILLOP,63.25
3,201903,DNEGOCIOS_EMPRESAS1,GRUPO ESPACIO URBANO S.A.C.,48691379.0,43530,NEWS,CCARRILLOP,63.25
4,201903,DNEGOCIOS_EMPRESAS1,GRUPO ESPACIO URBANO S.A.C.,48691408.0,43530,NEWS,CCARRILLOP,63.25
5,201903,DNEGOCIOS_EMPRESAS1,CONSORCIO MATERIALES NASCA S.A.C.,48751285.0,43539,ADDS,CCARRILLOP,63.25
6,201903,DNEGOCIOS_EMPRESAS1,CONSORCIO MATERIALES NASCA S.A.C.,48751286.0,43539,ADDS,CCARRILLOP,63.25
7,201903,DNEGOCIOS_EMPRESAS1,CONSORCIO MATERIALES NASCA S.A.C.,48751310.0,43539,ADDS,CCARRILLOP,63.25
8,201903,DNEGOCIOS_EMPRESAS1,CONSORCIO MATERIALES NASCA S.A.C.,48751324.0,43539,ADDS,CCARRILLOP,63.25
9,201903,DNEGOCIOS_EMPRESAS1,CONSORCIO MATERIALES NASCA S.A.C.,48751325.0,43539,ADDS,CCARRILLOP,63.25


In [13]:


#1. Carga de Data en Base de Datos

dbmanager.dbOperation('insert','tblEmpleados',ingresos) #ok
#dbmanager.dbOperation('insert','tblJerarquia',jerarquia) #ok
#dbmanager.dbOperation('insert','tblVentas',inar)#ok
#dbmanager.dbOperation('insert','tblDeacs',deacs) #ok

#dbmanager.dbOperation('insert','tblVentaSSAA',ventas_ssaa) ya no se usa
#dbmanager.dbOperation('insert','tblVentaSSAANew',ventas_ssaa) #ok
#dbmanager.dbOperation('insert','tblDeacSSAA',deacs_ssaa) FALTA, investigar
#dbmanager.dbOperation('insert','tblPaquetes',paquetes) #ok

#********************************************
#2.  Update de tablas de Ventas y Deacs (deben estar en memoria)

#dbmanager.dbOperation('update','Ceses', ceses)  #ok
#dbmanager.dbOperation('update','UpdatePuesto', nuevospuestos)

#dbmanager.dbOperation('update','Bolsas',bolsas) #no hubo
#dbmanager.dbOperation('update','SumVentaSSAA',ventas_ssaa) #ok
#dbmanager.dbOperation('update','Paquetes') #ok
#dbmanager.dbOperation('update','Unitarios', unitarios) #ok, se debe meter en los contratos de las ventas del mes pasado

#dbmanager.dbOperation('update','Actividad', actividad)
#dbmanager.dbOperation('update','Bases_GCP', basesgcp)

#3. Calculando el Gross Comisión y las Reversiones (Obligatorio al final de la carga)
#*************************************************
start_time = time.time()

#dbmanager.dbOperation('update','Gross_Comision') # ok
#dbmanager.dbOperation('update','Reversiones') # ok

time_consuming = (time.time() - start_time) # In minutes
print("--- %s seconds ---" % time_consuming)

# DEPRECATED:
#dbmanager.dbOperation('update','Bases_GCP_Quincenal',basesgcpquincenal) # actualizar con data quincenal,
#dbmanager.dbOperation('update','Bases_GCE',basesgce) # Usar a conveniencia,

INFO:    Setting database path to C:/Users/Johnny/OneDrive - Entel Peru S.A/SQLiteDB/mercado_empresas_db.sqlite


Los registros de la tabla tblEmpleados es 2 registros insertados 
--- 0.0 seconds ---
